In [1]:
import multiprocessing as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
inter_df = pd.read_csv('floor_info_maindata_inter_baseline_2.csv')

Wall time: 9.85 s


In [3]:
inter_df.shape

(6880841, 16)

In [30]:
inter_df.head()

,관리_건축물대장_PK,건물_명,층_구분,층_번호,층_번호_명,구조,기타_구조,주_용도,기타_용도,면적(㎡),주_부속_구분,면적_제외_여부,생성_일자,기준년월,지번주소,addr_split_len
0,11440-3279202,중동빌라,지상,1,1층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,57.62,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
1,11440-3279202,중동빌라,지상,2,2층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
2,11440-3279202,중동빌라,지상,3,3층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,67.20,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
3,11440-3279202,중동빌라,지상,4,4층,철근콘크리트구조,철근콘크리트구조,다세대주택,다세대주택,59.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4
4,11440-3279202,중동빌라,옥탑,1,옥탑 1층,철근콘크리트구조,철근콘크리트구조,다세대주택,계단실(연면적제외),7.68,주건축물,NaN,20141109,201412,서울특별시 마포구 중동 81-29,4


In [4]:
structure_df = inter_df[['지번주소', '기준년월', '구조']]

In [5]:
structure_df.head()

,지번주소,기준년월,구조
0,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
1,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
2,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
3,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조
4,서울특별시 마포구 중동 81-29,201412,철근콘크리트구조


In [6]:
structure_df = structure_df.sort_values(['지번주소', '기준년월']).reset_index(drop=True)

In [7]:
structure_df.head()

,지번주소,기준년월,구조
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조


In [8]:
structure_df.head(20)

,지번주소,기준년월,구조
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
5,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
6,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
7,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
8,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조
9,서울특별시 강남구 개포동 1163-4,201512,철근콘크리트구조


In [9]:
structure_df['구조'].value_counts()

철근콘크리트구조        6280233
벽돌구조             465277
철골철근콘크리트구조        77009
경량철골구조            18838
일반철골구조             8635
프리케스트콘크리트구조        8212
철골콘크리트구조           7881
일반목구조              4666
블록구조               4093
기타조적구조             3672
기타콘크리트구조           1213
기타강구조               432
기타구조                194
강파이프구조               73
라멘조                  36
석구조                  33
조적구조                 28
철골철근콘크리트합성구조         24
기타철골철근콘크리트구조          7
조립식판넬조                6
Name: 구조, dtype: int64

In [10]:
structure_df['count'] = structure_df.groupby(['지번주소', '기준년월']).transform('count')['구조']

In [12]:
structure_df.head(20)

,지번주소,기준년월,구조,count
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
5,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
6,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
7,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
8,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9
9,서울특별시 강남구 개포동 1163-4,201512,철근콘크리트구조,9


In [15]:
structure_df.dropna(inplace=True)

In [16]:
for structure in structure_df['구조'].unique():
    structure_df['구조_'+structure] = structure_df['구조'].apply(lambda x: 1 if x == structure else 0)

In [17]:
structure_df.head(20)

,지번주소,기준년월,구조,count,구조_철근콘크리트구조,구조_철골철근콘크리트구조,구조_벽돌구조,구조_프리케스트콘크리트구조,구조_경량철골구조,구조_일반철골구조,구조_블록구조,구조_철골콘크리트구조,구조_일반목구조,구조_기타조적구조,구조_기타강구조,구조_기타구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_강파이프구조,구조_기타콘크리트구조,구조_조립식판넬조,구조_조적구조,구조_라멘조,구조_철골철근콘크리트합성구조
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,서울특별시 강남구 개포동 1163-4,201512,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
for structure in tqdm(structure_df['구조'].unique()):
    structure_df['구조_'+structure+'_sum'] = structure_df[['지번주소', '기준년월', '구조_'+structure]].groupby(['지번주소', '기준년월']).transform('sum')['구조_'+structure]

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:36<00:00,  1.80s/it]


In [19]:
structure_df.head()

,지번주소,기준년월,구조,count,구조_철근콘크리트구조,구조_철골철근콘크리트구조,구조_벽돌구조,구조_프리케스트콘크리트구조,구조_경량철골구조,구조_일반철골구조,구조_블록구조,구조_철골콘크리트구조,구조_일반목구조,구조_기타조적구조,구조_기타강구조,구조_기타구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_강파이프구조,구조_기타콘크리트구조,구조_조립식판넬조,구조_조적구조,구조_라멘조,구조_철골철근콘크리트합성구조,구조_철근콘크리트구조_sum,구조_철골철근콘크리트구조_sum,구조_벽돌구조_sum,구조_프리케스트콘크리트구조_sum,구조_경량철골구조_sum,구조_일반철골구조_sum,구조_블록구조_sum,구조_철골콘크리트구조_sum,구조_일반목구조_sum,구조_기타조적구조_sum,구조_기타강구조_sum,구조_기타구조_sum,구조_기타철골철근콘크리트구조_sum,구조_석구조_sum,구조_강파이프구조_sum,구조_기타콘크리트구조_sum,구조_조립식판넬조_sum,구조_조적구조_sum,구조_라멘조_sum,구조_철골철근콘크리트합성구조_sum
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [20]:
for structure in tqdm(structure_df['구조'].unique()):
    structure_df['구조_'+structure+'_fraction'] = structure_df['구조_'+structure+'_sum'] / structure_df['count']

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 37.54it/s]


In [22]:
structure_df.head(100)

,지번주소,기준년월,구조,count,구조_철근콘크리트구조,구조_철골철근콘크리트구조,구조_벽돌구조,구조_프리케스트콘크리트구조,구조_경량철골구조,구조_일반철골구조,구조_블록구조,구조_철골콘크리트구조,구조_일반목구조,구조_기타조적구조,구조_기타강구조,구조_기타구조,구조_기타철골철근콘크리트구조,구조_석구조,구조_강파이프구조,구조_기타콘크리트구조,구조_조립식판넬조,구조_조적구조,구조_라멘조,구조_철골철근콘크리트합성구조,구조_철근콘크리트구조_sum,구조_철골철근콘크리트구조_sum,구조_벽돌구조_sum,구조_프리케스트콘크리트구조_sum,구조_경량철골구조_sum,구조_일반철골구조_sum,구조_블록구조_sum,구조_철골콘크리트구조_sum,구조_일반목구조_sum,구조_기타조적구조_sum,구조_기타강구조_sum,구조_기타구조_sum,구조_기타철골철근콘크리트구조_sum,구조_석구조_sum,구조_강파이프구조_sum,구조_기타콘크리트구조_sum,구조_조립식판넬조_sum,구조_조적구조_sum,구조_라멘조_sum,구조_철골철근콘크리트합성구조_sum,구조_철근콘크리트구조_fraction,구조_철골철근콘크리트구조_fraction,구조_벽돌구조_fraction,구조_프리케스트콘크리트구조_fraction,구조_경량철골구조_fraction,구조_일반철골구조_fraction,구조_블록구조_fraction,구조_철골콘크리트구조_fraction,구조_일반목구조_fraction,구조_기타조적구조_fraction,구조_기타강구조_fraction,구조_기타구조_fraction,구조_기타철골철근콘크리트구조_fraction,구조_석구조_fraction,구조_강파이프구조_fraction,구조_기타콘크리트구조_fraction,구조_조립식판넬조_fraction,구조_조적구조_fraction,구조_라멘조_fraction,구조_철골철근콘크리트합성구조_fraction
0,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,서울특별시 강남구 개포동 1163-4,201412,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,서울특별시 강남구 개포동 1163-4,201512,철근콘크리트구조,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
structure_df['구조_철근콘크리트구조_fraction'].describe()

count    6.880562e+06
mean     9.127500e-01
std      2.540157e-01
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: 구조_철근콘크리트구조_fraction, dtype: float64

In [25]:
cols_to_keep = [col for col in structure_df.columns if 'fraction' in col]
cols_to_keep

['구조_철근콘크리트구조_fraction',
 '구조_철골철근콘크리트구조_fraction',
 '구조_벽돌구조_fraction',
 '구조_프리케스트콘크리트구조_fraction',
 '구조_경량철골구조_fraction',
 '구조_일반철골구조_fraction',
 '구조_블록구조_fraction',
 '구조_철골콘크리트구조_fraction',
 '구조_일반목구조_fraction',
 '구조_기타조적구조_fraction',
 '구조_기타강구조_fraction',
 '구조_기타구조_fraction',
 '구조_기타철골철근콘크리트구조_fraction',
 '구조_석구조_fraction',
 '구조_강파이프구조_fraction',
 '구조_기타콘크리트구조_fraction',
 '구조_조립식판넬조_fraction',
 '구조_조적구조_fraction',
 '구조_라멘조_fraction',
 '구조_철골철근콘크리트합성구조_fraction']

In [26]:
fraction_df = structure_df[['지번주소', '기준년월'] + cols_to_keep]
print(fraction_df.shape)
fraction_df.head()

(6880562, 22)


,지번주소,기준년월,구조_철근콘크리트구조_fraction,구조_철골철근콘크리트구조_fraction,구조_벽돌구조_fraction,구조_프리케스트콘크리트구조_fraction,구조_경량철골구조_fraction,구조_일반철골구조_fraction,구조_블록구조_fraction,구조_철골콘크리트구조_fraction,구조_일반목구조_fraction,구조_기타조적구조_fraction,구조_기타강구조_fraction,구조_기타구조_fraction,구조_기타철골철근콘크리트구조_fraction,구조_석구조_fraction,구조_강파이프구조_fraction,구조_기타콘크리트구조_fraction,구조_조립식판넬조_fraction,구조_조적구조_fraction,구조_라멘조_fraction,구조_철골철근콘크리트합성구조_fraction
0,서울특별시 강남구 개포동 1163-4,201412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,서울특별시 강남구 개포동 1163-4,201412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,서울특별시 강남구 개포동 1163-4,201412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,서울특별시 강남구 개포동 1163-4,201412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,서울특별시 강남구 개포동 1163-4,201412,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
fraction_df.drop_duplicates(inplace=True)

In [28]:
fraction_df.shape

(703336, 22)

In [29]:
fraction_df.to_csv('./prepped_data/층별개요_구조_fraction.csv', index=False)